In [1]:
#pip install opencv-python


In [2]:
import tensorflow as tf
import time
from tensorflow.keras import Model, layers, applications, optimizers, callbacks
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.applications import DenseNet121
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import cv2
import os


In [3]:
# Load and preprocess the dataset
def load_and_preprocess_data(data_path):
    if not os.path.exists(data_path):
        raise FileNotFoundError(f"The file {data_path} does not exist.")

    data = pd.read_csv(data_path)
    if data.empty:
        raise ValueError(f"The file {data_path} is empty.")

    X = []
    y = []

    for i, row in data.iterrows():
        try:
            pixel_data = row['pixels']
            pixels = list(map(int, pixel_data.split(' ')))

            if len(pixels) != 48 * 48:
                print(f"Skipping row {i}: Pixel data length is {len(pixels)} instead of 48x48.")
                continue

            image = np.array(pixels).reshape(48, 48).astype(np.uint8)
            image = cv2.resize(image, (224, 224), interpolation=cv2.INTER_CUBIC)
            X.append(image)
            y.append(row['emotion'])
        except Exception as e:
            print(f"Error processing row {i}: {str(e)}")

    if not X or not y:
        raise ValueError("No valid data found in the CSV file.")

    X = np.array(X)
    y = np.array(y)

    X = np.expand_dims(X, axis=-1)
    X = np.repeat(X, 3, axis=-1)  # Convert grayscale to RGB
    X = X / 255.0  # Normalize the pixel values

    return X, y



In [4]:
# Custom Random Rotation
def random_rotation(image):
    return tf.image.rot90(image, k=tf.random.uniform([], minval=0, maxval=4, dtype=tf.int32))

# Custom Random Cutout
def random_cutout(image, mask_size=(40, 40)):
    h, w = image.shape[0], image.shape[1]
    mask_height, mask_width = mask_size

    top = tf.random.uniform([], 0, h - mask_height, dtype=tf.int32)
    left = tf.random.uniform([], 0, w - mask_width, dtype=tf.int32)

    cutout = tf.ones((mask_height, mask_width, 3), dtype=image.dtype) * 0  # Black mask
    mask = tf.image.pad_to_bounding_box(cutout, top, left, h, w)

    return image * (1 - mask)

In [5]:
# Enhanced Data Augmentation and Preprocessing
def preprocess_image(image, label):
    if image.shape[-1] != 3:
        image = tf.image.grayscale_to_rgb(image)

    image = tf.image.random_flip_left_right(image)
    image = tf.image.random_brightness(image, max_delta=0.2)
    image = tf.image.random_contrast(image, lower=0.8, upper=1.2)
    image = random_rotation(image)
    image = tf.image.random_saturation(image, lower=0.5, upper=1.5)
    image = tf.image.random_hue(image, max_delta=0.2)
    image = random_cutout(image)

    return image, label


In [6]:
# Create Dataset with Batch Size
def create_dataset(x, y, batch_size=32, is_training=True):
    dataset = tf.data.Dataset.from_tensor_slices((x, y))
    if is_training:
        dataset = dataset.map(preprocess_image, num_parallel_calls=tf.data.AUTOTUNE)
        dataset = dataset.shuffle(10000)
    dataset = dataset.batch(batch_size).prefetch(tf.data.AUTOTUNE)
    return dataset



In [7]:
# Build Model
def build_model(num_classes, dropout_rate=0.5, dense_units=512):
    base_model = DenseNet121(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

    # Unfreeze the last few layers of the base model
    for layer in base_model.layers[-30:]:
        layer.trainable = True

    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(dense_units, activation='relu')(x)
    x = Dropout(dropout_rate)(x)
    predictions = Dense(num_classes, activation='softmax')(x)

    model = Model(inputs=base_model.input, outputs=predictions)

    return model



In [8]:
# Plot Training History
def plot_history(history):
    plt.figure(figsize=(12, 4))

    plt.subplot(1, 2, 1)
    plt.plot(history.history['accuracy'], label='Train Accuracy')
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
    plt.title('Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()

    plt.subplot(1, 2, 2)
    plt.plot(history.history['loss'], label='Train Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.title('Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()

    plt.tight_layout()
    plt.savefig('training_history.png')
    plt.close()



In [9]:
# Plot Confusion Matrix
def plot_confusion_matrix(y_true, y_pred, classes):
    cm = confusion_matrix(y_true, y_pred)
    plt.figure(figsize=(10, 8))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=classes, yticklabels=classes)
    plt.title('Confusion Matrix')
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.tight_layout()
    plt.savefig('confusion_matrix.png')
    plt.close()



In [11]:

import ssl
ssl._create_default_https_context = ssl._create_unverified_context

# Main Training Function
def main():
    # Hyperparameters
    batch_size = 32
    dropout_rate = 0.5
    dense_units = 512
    initial_learning_rate = 0.001
    epochs = 50

    # Load and preprocess data
    data_path = 'ckextended.csv'
    print("Loading data...")
    try:
        X, y = load_and_preprocess_data(data_path)
    except Exception as e:
        print(f"Error loading data: {str(e)}")
        return

    # Define emotion dictionary
    emotion_dict = {0: 'Angry', 1: 'Disgust', 2: 'Fear', 3: 'Happy', 4: 'Sad', 5: 'Surprise', 6: 'Neutral'}

    # Filter out any samples where the label is not in the range 0-6
    valid_indices = [i for i, label in enumerate(y) if label in emotion_dict]
    X = np.array([X[i] for i in valid_indices])
    y = np.array([y[i] for i in valid_indices])

    # Split the dataset
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

    # Create datasets
    train_dataset = create_dataset(X_train, y_train, batch_size=batch_size)
    val_dataset = create_dataset(X_val, y_val, batch_size=batch_size, is_training=False)
    test_dataset = create_dataset(X_test, y_test, batch_size=batch_size, is_training=False)

    # Build model
    num_classes = len(emotion_dict)
    model = build_model(num_classes, dropout_rate=dropout_rate, dense_units=dense_units)

    # Compile model
    optimizer = Adam(learning_rate=initial_learning_rate)
    model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

  # Callbacks
    early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
    model_checkpoint = ModelCheckpoint('best_model.keras', monitor='val_loss', save_best_only=True, mode='min')
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=1e-6)

    # Train model
    # Measure training time
    start_time = time.time()
    history = model.fit(train_dataset, validation_data=val_dataset, epochs=50, callbacks=[early_stopping, model_checkpoint, reduce_lr])
    end_time = time.time()
    print(f"Training Time: {end_time - start_time:.2f} seconds")

    # Plot training history
    plot_history(history)

    # Measure evaluation time
    start_time = time.time()
    test_loss, test_accuracy = model.evaluate(test_dataset)
    end_time = time.time()
    print(f"Evaluation Time: {end_time - start_time:.2f} seconds")
    print(f'Test Accuracy: {test_accuracy * 100:.2f}%')
    # Generate predictions for confusion matrix
    y_pred = []
    y_true = []

    for images, labels in test_dataset:
        predictions = model.predict(images)
        y_pred.extend(np.argmax(predictions, axis=1))
        y_true.extend(labels.numpy())

    # Plot confusion matrix
    plot_confusion_matrix(y_true, y_pred, list(emotion_dict.values()))

    # Print classification report
    print("\nClassification Report:")
    print(classification_report(y_true, y_pred, target_names=list(emotion_dict.values())))

    # Save the final model
    model.save('final_model.keras')

if __name__ == '__main__':
    main()

Loading data...
29084464/29084464 ━━━━━━━━━━━━━━━━━━━━ 150s 5us/step
Epoch 1/50
18/18 ━━━━━━━━━━━━━━━━━━━━ 112s 5s/step - accuracy: 0.4651 - loss: 2.0131 - val_accuracy: 0.6483 - val_loss: 3.6747 - learning_rate: 0.0010
Epoch 2/50
18/18 ━━━━━━━━━━━━━━━━━━━━ 96s 5s/step - accuracy: 0.6479 - loss: 1.2472 - val_accuracy: 0.6483 - val_loss: 5.5249 - learning_rate: 0.0010
Epoch 3/50
18/18 ━━━━━━━━━━━━━━━━━━━━ 93s 5s/step - accuracy: 0.6382 - loss: 1.1878 - val_accuracy: 0.6483 - val_loss: 4.5051 - learning_rate: 0.0010
Epoch 4/50
18/18 ━━━━━━━━━━━━━━━━━━━━ 95s 5s/step - accuracy: 0.7198 - loss: 0.9327 - val_accuracy: 0.7241 - val_loss: 2.2517 - learning_rate: 0.0010
Epoch 5/50
18/18 ━━━━━━━━━━━━━━━━━━━━ 94s 5s/step - accuracy: 0.7867 - loss: 0.7208 - val_accuracy: 0.7172 - val_loss: 2.3493 - learning_rate: 0.0010
Epoch 6/50
18/18 ━━━━━━━━━━━━━━━━━━━━ 102s 6s/step - accuracy: 0.8145 - loss: 0.6366 - val_accuracy: 0.6414 - val_loss: 3.7634 - learning_rate: 0.0010
Epoch 7/50
18/18 ━━━━━━━━━━━━

In [16]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix

def plot_confusion_matrix(y_true, y_pred, labels):
    """
    Plots the confusion matrix using true and predicted labels.

    Parameters:
    - y_true: array-like of shape (n_samples,), Ground truth (correct) target values.
    - y_pred: array-like of shape (n_samples,), Estimated targets as returned by a classifier.
    - labels: list of string labels for the confusion matrix.

    Returns:
    - None
    """
    # Generate confusion matrix
    cm = confusion_matrix(y_true, y_pred)
    
    # Plot confusion matrix
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=labels, yticklabels=labels)
    plt.xlabel('Predicted Label')
    plt.ylabel('True Label')
    plt.title('Confusion Matrix')
    plt.show()


In [18]:
emotion_labels = ['Anger', 'Disgust', 'Happy', 'Sad', 'Surprise', 'Neutral', 'Fear']
# Assuming y_true and y_pred have been populated after model predictions
plot_confusion_matrix(y_true, y_pred, emotion_labels)


NameError: name 'y_true' is not defined